In [1]:
from scipy.stats import ttest_ind, normaltest, shapiro

split_n = 0
splits = {}
with open("ALL_classification_results.log", "r") as f:
    lines = f.readlines()
    for line in lines:
        split_line = line.split()
        if split_line[0] == "Split":
            split_n = int(split_line[1])
            splits[split_n] = {}
        elif split_line[0] == "BERT":
            if split_line[1] == "short":
                splits[split_n]["BERT_short"] = (float(split_line[5][:-1]))
            else:
                splits[split_n]["BERT_long"] = (float(split_line[5][:-1]))
        elif split_line[0] == "CNN":
            splits[split_n]["CNN"] = (float(split_line[3][:-1]))

        elif split_line[0] == "GBC":
            splits[split_n]["GBC"] = (float(split_line[3][:-1]))

In [3]:
import pandas as pd
dt = pd.DataFrame(splits)

/tmp/ipykernel_1852504/227332269.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


CNN_acc = []
with open("CNN_out.log", "r") as f:
    lines = f.readlines()
    for line in lines:
        split_line = line.split()
        CNN_acc.append(float(split_line[1][:-1]))
dt.loc["CNN"] = CNN_acc

In [4]:
dt

,0,1,2,3,4,5
BERT_short,0.627233,0.608641,0.645859,0.634527,0.601626,0.647618
BERT_long,0.639330,0.583843,0.673088,0.641866,0.605586,0.670955
CNN,0.599594,0.605412,0.592102,0.561949,0.583062,0.577744
GBC,0.568598,0.567260,0.566326,0.567128,0.562818,0.565071


In [5]:
_, p_bert_short = shapiro(dt.loc["BERT_short"])
_, p_bert_long = shapiro(dt.loc["BERT_long"])
_, p_cnn = shapiro(dt.loc["CNN"])
#_, p_gbc = shapiro(dt.loc["GBC"])
print("Normality not rejected:", p_bert_short > 0.05, p_bert_long > 0.05, p_cnn > 0.05)#, p_gbc > 0.05)

Normality not rejected: True True True


### In case normality assumption is met

In [6]:
#BERT vs CNN
ttest_ind((dt.loc["BERT_short"] + dt.loc["BERT_long"])/2, dt.loc["CNN"])

TtestResult(statistic=3.532918593580584, pvalue=0.0054203650189217955, df=10.0)

In [7]:
#BERT vs GBC
ttest_ind((dt.loc["BERT_short"] + dt.loc["BERT_long"])/2, dt.loc["GBC"])

TtestResult(statistic=5.941274493256746, pvalue=0.00014290376207360702, df=10.0)

In [8]:
#CNN vs GBC
ttest_ind(dt.loc["CNN"], dt.loc["GBC"])

TtestResult(statistic=3.139432512033814, pvalue=0.010520321364425248, df=10.0)